# In this notebook:

* Load graph

In [ ]:
from graph_tool.all import *
import matplotlib.pyplot as plt
import numpy as np
import os
import json

## Load graph and pos2node

In [ ]:
base_path = "../outputs/path_92699"

In [ ]:
pos2node = np.load(base_path+"_pos2node.npy")

In [ ]:
plt.figure(figsize=(20,10))
plt.imshow(pos2node[70:180, 10:250])
plt.show()

In [ ]:
# load graph itself
graph = load_graph(base_path + "_graph.xml.gz")

In [ ]:
all_costs = graph.ep.weight
cost_props = list()
classes = ["env", "urban"]
for props in classes:
    cost_props.append(eval("graph.ep."+props))

In [ ]:
# test: all costs if combination of others
print(cost_props[0].get_array()[10], cost_props[1].get_array()[10], all_costs.get_array()[10])

In [ ]:
0.333333*0.4 + 0.333333*0.35

In [ ]:
# load infos
with open(base_path+"_infos.json", "r") as infile:
    infos_path = json.load(infile)

In [ ]:
## beginning: 5 293 756 edges
# start cells: [408  62] dest cells: [235 601]        
source = 16749 # 58730 
dest = 3557 # 12160

## Randomly delete edges:

In [ ]:
def remove_edges(graph, remove_percent = 0.3):
    n_edges = len(list(graph.edges()))
    print("prev number edges", n_edges)
    remove_ind = int(n_edges * remove_percent)
    print("removing ", remove_ind, "edges")
    remove = np.random.permutation(np.arange(n_edges))[:remove_ind]
    remove_arr = np.zeros(n_edges)
    remove_arr[remove] = 1
    # del graph.properties[("e", "remove_property")]
    remove_property = graph.new_edge_property("float")
    remove_property.a = remove_arr
    remove_labeled_edges(graph, remove_property)
    print("now number edges:", len(list(graph.edges())))

In [ ]:
remove_edges(graph, remove_percent = 0.5)

In [ ]:
# USE SET FILTERS FUNCTION
# set_filters(self, eprop, vprop, inverted_edges=False, inverted_vertices=False)[source]¶

In [ ]:
# To also reduce property maps
keep = np.abs(1-remove_arr)
arr = all_costs.get_array()
arr_keep = arr*keep
print(len(arr_keep))
arr_keep = arr_keep[arr_keep>0]
print(len(arr_keep))

In [ ]:
# edges = graph.get_edges()
# v1, v2 = edges[2000]
all_costs[graph.edge(graph.vertex(v1), graph.vertex(v2))]

In [ ]:
edges = graph.get_edges()
v1, v2 = edges[2000]
print(v1,v2)
all_costs[graph.edge(graph.vertex(v1), graph.vertex(v2))]

In [ ]:
print("average path costs for original:", np.mean(np.asarray(infos_path["edgecosts"]), axis=0))
# not equal to costs computed here weil edge costs vs node costs

### Compute probability that edges are kept

In [ ]:
m = 483010 # n_edges
k = 50 # length of path
p = 0.005 # probability remove
def prob_kept(m,k,p):
    n_rem = int(m*p)
    print(m-k, m, n_rem)
    # prob = binom(m-k, n_rem) / binom(m, n_rem) # number of sets to draw with 
    zaehler = np.arange(m-n_rem, m-n_rem-k, -1)
    nenner = np.arange(m,m-k, -1)
    prob = np.product(zaehler/nenner)
    print("probability to keep the shortest path edges", prob)
    return prob
_ = prob_kept(m,k,p)

### Baseline path: path without weights

In [ ]:
vertices_path, edges_path = get_path(graph_red, None)
path, path_costs = convert_path(graph_red, vertices_path, edges_path, pos2node)

In [ ]:
path = np.asarray(path)
plt.plot(path[:,0], path[:,1])
plt.show()
print("Baseline path:", "length:", len(path), "costs:", np.sum(path_costs,axis=0))

### Remove edges and compute path

In [ ]:
# same procedure as for pareto fronteir, just vary the number of removed edges instead
rem = np.logspace(-5, -1, 10, base=3)
print(rem)
pareto = []
paths = list()
for r in rem:
    if r==0:
        graph_red = graph
    else:
        graph_red = graph.copy()
        remove_edges(graph_red, remove_percent = r)
    all_costs = graph_red.ep.weight
    vertices_path, edges_path = get_path(graph_red, all_costs)
    path, path_costs = convert_path(graph_red, vertices_path, edges_path, pos2node)
    print("removed:", r, "cost sum", np.sum(path_costs, axis=0), "path length", len(path))
    pareto.append(np.sum(path_costs, axis=0))
    paths.append(path)

## All shortest paths:
* all_paths function of graph-tool does DFS --> not useful
* all_shortest_paths of graph-tool returns only the unique single path in our case

### Round edge costs --> more shortest paths 

In [ ]:
costs_rounded = graph.new_edge_property("int")
costs_rounded.a = (cost_props[0].get_array()*10).astype(int)

In [ ]:
max_paths = 100
paths = []
rem = []
for i,vertices_path in enumerate(all_shortest_paths(graph, graph.vertex(source),
                    graph.vertex(dest), weights=costs_rounded,
                    negative_weights=True)):
    rem.append(i)
    # edges_path = []
    path, _ = convert_path(graph, vertices_path, [], pos2node)
    paths.append(path)
    if i>max_paths:
        break
print("number of paths:", i)

## Shortest path

In [ ]:
def get_path(graph, all_costs):
    vertices_path, edges_path = shortest_path(
                    graph,
                    graph.vertex(source),
                    graph.vertex(dest),
                    weights=all_costs,
                    negative_weights=True
                )
    return vertices_path, edges_path
def convert_path(graph, vertices_path, edges_path, pos2node):
    path = []
    for v in vertices_path:
        pos_x, pos_y = np.where(pos2node==graph.vertex_index[v])
        assert len(pos_x)==1, "posx{}".format(pos_x)
        path.append([pos_x[0], pos_y[0]])

    path_costs = []
    for e in edges_path:
        costs = [props[e] for props in cost_props]
        path_costs.append(costs)
    return path, path_costs

In [ ]:
len(all_costs.get_array())

In [ ]:
path, path_costs = get_path(graph, all_costs, pos2node)

In [ ]:
path_orig = infos_path["path_cells"]

In [ ]:
len(path_orig)

## Pareto frontier

In [ ]:
rem = np.arange(0,1.1, 0.1)
pareto = []
paths = list()
all_costs = graph.ep.weight
for w in rem:
    all_costs.a = cost_props[0].get_array()*w + cost_props[1].get_array() * (1-w)
    vertices_path, edges_path = get_path(graph, all_costs)
    path, path_costs = convert_path(graph, vertices_path, edges_path, pos2node)
    print(w, 1-w, np.sum(path_costs, axis=0))
    pareto.append(np.sum(path_costs, axis=0))
    paths.append(path)

In [ ]:
pareto = np.asarray(pareto)
color=plt.cm.rainbow(np.linspace(0,1,len(pareto)))
plt.subplot(1,2,1)
plt.scatter(pareto[:,0], pareto[:,1], c=color)
plt.xlabel(classes[0])
plt.ylabel(classes[1])
plt.legend()

plt.subplot(1,2,2)
plt.scatter(pareto[:,0], pareto[:,1], label=rem, c=color)
plt.xlabel(classes[0])
plt.ylabel(classes[1], fontsize=15)
plt.legend()
plt.show()

In [ ]:
e = find_edge(graph, graph.edge_index, 33)
all_costs[e[0]]

In [ ]:
color=iter(plt.cm.rainbow(np.linspace(0,1,len(paths))))
plt.figure(figsize=(20,10))
for i,p in enumerate(paths):
    p_arr = np.array(p)
    c = next(color)
    plt.plot(p_arr[:,1], p_arr[:,0], label = rem[i], c=c)
    # print("path length:", len(p))
plt.legend( title="Weight of "+classes[0]+"costs")
plt.savefig("dropout_path.png")
plt.show()

## Path to distance corridor

In [ ]:
from power_planner.utils.utils import bresenham_line
from scipy.ndimage.morphology import binary_dilation
from scipy.spatial.distance import cdist
import time

In [ ]:
def get_path_lines(cost_shape, paths):
    path_dilation = np.zeros(cost_shape)
    for path in paths:
        # iterate over path nodes
        for i in range(len(path)-1):
            line = bresenham_line(*path[i], *path[i+1])
            # print(line)
            for (j,k) in line:
                path_dilation[j,k] = 1
    return path_dilation

In [ ]:
def dilation_dist(path_dilation):
    """
    path_dilation: binary array with zeros everywhere except for path locations
    """
    saved_arrs = [path_dilation]
    # compute number of iterations: maximum distance of pixel to line
    x_coords, y_coords = np.where(path_dilation)
    x_len, y_len = path_dilation.shape
    # print([np.min(x_coords), x_len- np.max(x_coords), np.min(y_coords), y_len- np.max(y_coords)])
    n_iters = max([np.min(x_coords), x_len- np.max(x_coords), np.min(y_coords), y_len- np.max(y_coords)])
    # dilate
    for k in range(n_iters):
        path_dilation = binary_dilation(path_dilation)
        saved_arrs.append(path_dilation)
    saved_arrs = np.sum(np.array(saved_arrs), axis=0)
    return saved_arrs

In [ ]:
def cdist_dist(path_dilation):
    saved_arrs = np.zeros(path_dilation.shape)
    x_len, y_len = path_dilation.shape
    xa = np.array([[i,j] for i in range(x_len) for j in range(y_len)])
    xb = np.swapaxes(np.vstack(np.where(path_dilation>0)),1,0)
    print(xa.shape, xb.shape)
    all_dists = cdist(xa,xb)
    print(all_dists.shape)
    out = np.min(all_dists, axis = 1)
    k=0
    for i in range(x_len):
        for j in range(y_len):
            saved_arrs[i,j] = out[k]
            k+=1
    return saved_arrs

In [ ]:
path_dilation = get_path_lines(pos2node.shape, paths)
tic = time.time()
saved_arrs = dilation_dist(path_dilation)
print("time dilation:",time.time()-tic)

In [ ]:
tic = time.time()
saved_arrs = cdist_dist(path_dilation)
print("time cdist:",time.time()-tic)

In [ ]:
plt.figure(figsize=(20,10))
plt.imshow(saved_arrs[70:180, 10:250])
plt.colorbar()
plt.show()

# Tests

In [ ]:
# make sure that edge costs are just the nodecosts means
edgecosts= np.asarray(infos_path["edgecosts"])
for p in range(len(path_costs)-1):
    costmean = edgecosts[p] + edgecosts[p+1]
    print(path_costs[p], 0.5*costmean)

In [ ]:
print(infos_path["path_cells"])

In [ ]:
print("Only included ", len(list(graph.vertices()))/(250*550), "% of the actual raster cells")

In [ ]:
len(list(graph.edges())) # --> actually 80 neighbors on average (65000*80 = 5200000)